# MlFlow and PyTorch MNIST 

## Imports

In [43]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

import mlflow
from mlflow import pytorch

## Network 

In [44]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
    

## Train and test functions

In [45]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args["log_interval"] == 0:
            # log loss metric
            mlflow.log_metric("train_loss_item", loss.item())
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    mlflow.log_metric("test_loss", test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    

## Main

In [46]:
def main(params=None):
    # Training settings
    args = params
    
    with mlflow.start_run():
        # use mlflow to log params
        mlflow.log_params(args)
        
        use_cuda = not args["no_cuda"] and torch.cuda.is_available()
    
        torch.manual_seed(args["seed"])
    
        device = torch.device("cuda" if use_cuda else "cpu")
        # log device
        mlflow.log_param("device", device)
    
        kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
        train_loader = torch.utils.data.DataLoader(
            datasets.MNIST('../data', train=True, download=True,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ])),
            batch_size=args["batch_size"], shuffle=True, **kwargs)
        test_loader = torch.utils.data.DataLoader(
            datasets.MNIST('../data', train=False, transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ])),
            batch_size=args["test_batch_size"], shuffle=True, **kwargs)
    
        model = Net().to(device)
        optimizer = optim.Adadelta(model.parameters(), lr=args["lr"])
    
        scheduler = StepLR(optimizer, step_size=1, gamma=args["gamma"])
        for epoch in range(1, args["epochs"] + 1):
            train(args, model, device, train_loader, optimizer, epoch)
            test(model, device, test_loader)
            # save checkpoint
            pytorch.save_model(model, "models/mnist_cnn_epoch_{}".format(epoch))
            scheduler.step()
    
        if args["save_model"]:
            mlflow.log_model(model.state_dict(), "mnist_cnn_final")

In [47]:
PARAMS = {
    "batch_size": 128,
    "test_batch_size": 1000,
    "epochs": 3,
    "lr": 1.0,
    "gamma": 0.7,
    "no_cuda": True,
    "seed": 1,
    "log_interval": 10,
    "save_model": True
}
main(params=PARAMS)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.296643
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.399460
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.662251
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.717085
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.464298
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.361052
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.291304
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.413936
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.148118
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.260617
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.237537
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.279195


KeyboardInterrupt: 